In [1]:
1

1

In [2]:
import torch
import math
import torch.nn as nn
from typing_extensions import Annotated


class PositionalEncoding(nn.Module):
    def __init__(self,
                 max_len: Annotated[int, "It means how many no of words are there in a sequence"],
                 d_model: Annotated[int, "It tells in how many dimension each and every word represents"]) -> None:
        super().__init__()
        positional_encoder = torch.zeros(
            max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(
            1).float()  # (max_len, 1)

        div_term = torch.exp(torch.arange(0, d_model, 2).float(
        ) * (-math.log(10000.0) / d_model))  # (d_model//2,)

        positional_encoder[:, 0::2] = torch.sin(position * div_term)
        positional_encoder[:, 1::2] = torch.cos(position * div_term)

        # Shape it to (1, max_len, d_model) for broadcasting with input: (batch_size, seq_len, d_model)
        positional_encoder = positional_encoder.unsqueeze(0)

        self.register_buffer("positional_encoder", positional_encoder)

    def forward(self, input_data: torch.Tensor) -> torch.Tensor:
        """
        input_data: shape (batch_size, seq_len, d_model)
        returns: same shape with positional encoding added
        """
        seq_len = input_data.size(1)
        return input_data + self.positional_encoder[:, :seq_len]


In [4]:
import math
import torch
import torch.nn as nn
from typing import List, Annotated, Union


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: Annotated[int, "No of self attention needed"],
                 embed_dim: Annotated[int, "dimension of each word"],
                 bias: Annotated[bool, "Required bias during trining"] = False,) -> None:
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim % num_heads != 0"
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.head_dim = self.embed_dim // self.num_heads
        self.wq = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.wk = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.wv = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.output_projection = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(self,
                q_input: Annotated[torch.Tensor, "batch of data from the input data"],
                k_input: Union[torch.Tensor, None] = None,
                v_input: Union[torch.Tensor, None] = None,
                mask: Annotated[bool, "normal MHA or masked MHA?"] = False) -> torch.Tensor:
        batch = q_input.size(0)

        # Below code make sures algorithm is self attention not cross attention
        if k_input is None:
            k_input = q_input
        if v_input is None:
            v_input = q_input

        q = self.wq(q_input)
        k = self.wk(k_input)
        v = self.wv(v_input)

        # Sequence length can be diffrent for input data and output data
        T_q, T_k = q_input.size(1), k_input.size(1)

        # Split the q, k, v(embed_dim) dimension as (num_head, embed_dim / num_head)
        q = q.reshape(batch, T_q, self.num_heads,
                      self.head_dim).transpose(1, 2)
        k = k.reshape(batch, T_k, self.num_heads,
                      self.head_dim).transpose(1, 2)
        v = v.reshape(batch, T_k, self.num_heads,
                      self.head_dim).transpose(1, 2)

        # Calculate Attention
        k_transpose = k.transpose(-2, -1)  # (b, k, d) (b, d, p) = (b, k, p)
        score = (q @ k_transpose) / math.sqrt(self.head_dim)
        mask = torch.triu(torch.ones(T_q, T_k), diagonal=1).bool(
        ) if mask else torch.zeros(T_q, T_k).bool()

        # Anyhow broadcasting works no need of unsqeeze but its good practice to
        # avoid broadcasting in Attentions, but clearly this step is optional
        mask = mask.unsqueeze(0).unsqueeze(0)
        score = score.masked_fill(mask, float("-inf"))
        attention_score = torch.softmax(score, dim=-1)
        attention = attention_score @ v

        # concat output of all heads
        attention = attention.transpose(1, 2)

        # Attention should have sequence of length = output sequence length.
        attention = attention.reshape(batch, T_q, self.embed_dim)

        # Since they are simple concatination to acutally mix all heads details we need a linear layer

        mha_output = self.output_projection(attention)
        return mha_output


In [5]:
import math
import torch
import torch.nn as nn

class LayerNormalization(nn.Module):
    def __init__(self, embed_dim:int, eps:float = 1e-9) -> None:
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(embed_dim)).float()
        self.beta = nn.Parameter(torch.ones(embed_dim)).float()
        self.eps = eps

    def forward(self, input_data:torch.Tensor) -> torch.Tensor:
        # Assume input dim(2, 3, 6)
        mean = torch.mean(input_data, dim=-1, keepdim=True) # (2, 3, 1)
        std = torch.std(input_data, dim=-1, keepdim=True) # (2, 3, 1)

        # to normalize (2, 3, 6) - (2, 3, 1) = (2, 3, 6) due to broadcasting
        normalized_input_data = (input_data - mean) / (std + self.eps)

        # some weights do not require normalized output so alpha learnable parameter is introduced 
        return self.alpha * normalized_input_data + self.beta
        

In [6]:
import math
import torch
import torch.nn as nn
from typing import List, Annotated

class FeedForward(nn.Module):
    def __init__(self, embed_dim:int, hidden_dim:int, dropout:float = 0.1,  bias:bool = True):
        super().__init__()
        self.w1 = nn.Linear(embed_dim, hidden_dim, bias=bias)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.w2 = nn.Linear(hidden_dim, embed_dim, bias=bias)

    def forward(self, input_data:torch.Tensor) -> torch.Tensor:
        layer = self.w1(input_data)
        output = self.relu(layer)
        dropout = self.dropout(output)
        layer = self.w2(dropout)
        
        return layer


In [23]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim:int, hidden_dim:int,num_heads:int, dropout = 0.1, bias:bool = False):
        super().__init__()
        self.mmha = MultiHeadAttention(num_heads=num_heads, embed_dim=embed_dim)
        self.cmha = MultiHeadAttention(num_heads=num_heads, embed_dim=embed_dim)
        self.ln1 = LayerNormalization(embed_dim=embed_dim)
        self.ln2 = LayerNormalization(embed_dim=embed_dim)
        self.ln3 = LayerNormalization(embed_dim=embed_dim)
        self.ffn = FeedForward(embed_dim=embed_dim, hidden_dim=hidden_dim, dropout=dropout, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x:torch.Tensor, encoder_output) -> torch.Tensor:
        # Multi-head attention + residual + norm
        x_mha = self.mmha(x, mask = True)
        x = self.ln1(x + self.dropout(x_mha))

        # Cross_Attention
        x_cmha = self.cmha(q_input=x, k_input=encoder_output, v_input=encoder_output)
        x = self.ln2(x + self.dropout(x_cmha))

        # Feedforward + residual + norm
        x_ffn = self.ffn(x)
        x = self.ln3(x + self.dropout(x_ffn))
        return x



class Decoder(nn.Module):
    def __init__(self, Nx, embed_dim, num_heads, ff_hidden_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            DecoderBlock(embed_dim, ff_hidden_dim, num_heads, dropout)
            for _ in range(Nx)
        ])
    
    def forward(self, x: torch.Tensor, encoder_output: torch.Tensor) -> torch.Tensor:
        for layer in self.layers:
            x = layer(x, encoder_output)
        return x



### Lets build Decoder block SOTA

1) lets suppose consider a output sentences of length 100 i.e. (100, 20, 10)

In [17]:
decoder_input = torch.arange(100 * 20 * 8).reshape(100, 20, 8).float()
encoder_output = torch.arange(100 * 10 * 8).reshape(100, 10, 8).float()

In [20]:
pe = PositionalEncoding(20, 8)

In [21]:
decoder_input = pe(decoder_input)

In [26]:
dc = Decoder(Nx=8,
             embed_dim=8,
             ff_hidden_dim=1024,
             dropout=0.1,
             num_heads=2
             )

decoder_output = dc(decoder_input, encoder_output)

In [27]:
decoder_output.shape

torch.Size([100, 20, 8])